<a href="https://colab.research.google.com/github/NoriaYousufi/Brain-Tumor-Detection/blob/main/Brain_Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import Libaries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split

from keras.utils import normalize, to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

###Load the data

In [3]:
image_directory='/content/drive/MyDrive/Datasets'
no_tumor_images=os.listdir(image_directory+ '/no')
yes_tumor_images=os.listdir(image_directory+ '/yes')

print('No Tumor: ', len(no_tumor_images))
print('Tumor: ',len(yes_tumor_images))

No Tumor:  1500
Tumor:  1500


In [15]:
dataset = []
label = []
INPUT_SIZE = 64

In [16]:
# Seperate images
# No tumor
for i, image_name in enumerate(no_tumor_images):
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(image_directory, 'no', image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image

        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)  # Assuming 0 represents images without tumors


In [17]:
for i, image_name in enumerate(yes_tumor_images):
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(image_directory, 'yes', image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image

        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)  # Assuming 1 represents images with tumors


In [18]:
print(len(dataset))
print(len(label))

3000
3000


In [19]:
dataset = np.array(dataset)
label = np.array(label)

###Splitting training and test data

In [20]:
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [21]:
print(x_train.shape)
print(y_train.shape)

(2400, 64, 64, 3)
(2400,)


In [22]:
print(x_test.shape)
print(y_test.shape)

(600, 64, 64, 3)
(600,)


###Normalize data

In [23]:
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

###CNN model

In [24]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [25]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))